In [2]:
pip install prettytable


In [4]:
!apt-get update
!apt-get install -y traceroute


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 Packages [77.3 kB]
Get:13 http://security

In [5]:
import json
import re
import subprocess
import sys
from urllib import request
from prettytable import PrettyTable

IP_PATTERN = re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
UNKNOWN_HOST_MSG = 'traceroute: unknown host'
TRACEROUTE_HEADER = 'traceroute to'
TIMEOUT_LINE = '* * *'


def запуск_traceroute(хост):
    процесс = subprocess.Popen(['traceroute', хост], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    return процесс.stdout.readline


def получить_инфо_ip(ip_адрес):
    url = f'http://ipinfo.io/{ip_адрес}/json'
    with request.urlopen(url) as ответ:
        return json.load(ответ)


def собрать_ip_маршрута(адрес):
    ip_маршрут = []
    начальный_ip = ""
    таймаут_счётчик = 0

    for строка in iter(запуск_traceroute(адрес), b""):
        строка = строка.decode('cp866')[4:]

        if UNKNOWN_HOST_MSG in строка:
            print(UNKNOWN_HOST_MSG)
            return

        elif TRACEROUTE_HEADER in строка:
            print(строка)
            начальный_ip = IP_PATTERN.findall(строка)[0]

        elif TIMEOUT_LINE in строка:
            if таймаут_счётчик == 2:
                break
            таймаут_счётчик += 1
            continue

        найденные_ip = IP_PATTERN.findall(строка)

        if not найденные_ip:
            return ip_маршрут

        текущий_ip = найденные_ip[0]

        if текущий_ip == начальный_ip:
            break

        ip_маршрут.append(текущий_ip)
        print(строка, 'ip = [', текущий_ip, ']')
        таймаут_счётчик = 0

    return ip_маршрут


def создать_таблицу(ip_список):
    таблица = PrettyTable(['№', 'IP', 'AS', 'Провайдер', 'Страна'])

    for номер, ip_ in enumerate(ip_список, start=1):
        инфо = получить_инфо_ip(ip_)
        print(инфо)

        организация = инфо.get('org') or '*'
        asn = организация.split()[0] if организация != '*' else '*'
        провайдер = ' '.join(организация.split()[1:]) if организация != '*' else '*'
        страна = инфо.get('country') or '*'

        таблица.add_row([
            номер,
            инфо.get('ip') or '*',
            asn,
            провайдер,
            страна
        ])

    return таблица


def точка_входа():
    адрес = input('Введите доменное имя или IP адрес: ')
    ip_список = собрать_ip_маршрута(адрес)
    if ip_список:
        print(создать_таблицу(ip_список))


if __name__ == '__main__':
    точка_входа()


Введите доменное имя или IP адрес: 91.109.200.200
eroute to 91.109.200.200 (91.109.200.200), 30 hops max, 60 byte packets
 ip = [ 91.109.200.200 ]
172.28.0.1 (172.28.0.1)  0.059 ms  0.012 ms  0.013 ms
 ip = [ 172.28.0.1 ]
172.253.51.75 (172.253.51.75)  42.905 ms  43.254 ms  42.806 ms
 ip = [ 172.253.51.75 ]
192.178.241.180 (192.178.241.180)  41.954 ms 192.178.241.118 (192.178.241.118)  44.874 ms 192.178.241.180 (192.178.241.180)  41.927 ms
 ip = [ 192.178.241.180 ]
msk-ix.atlex.ru (195.208.208.153)  43.098 ms  43.017 ms  43.001 ms
 ip = [ 195.208.208.153 ]
msk-f702.rudc.cloud (91.109.200.200)  44.752 ms !X  44.732 ms !X  44.945 ms !X
 ip = [ 91.109.200.200 ]
{'ip': '91.109.200.200', 'hostname': 'msk-f702.rudc.cloud', 'city': 'Moscow', 'region': 'Moscow', 'country': 'RU', 'loc': '55.7522,37.6156', 'org': 'AS199669 Okay-Telecom Ltd.', 'postal': '101000', 'timezone': 'Europe/Moscow', 'readme': 'https://ipinfo.io/missingauth'}
{'ip': '172.28.0.1', 'bogon': True}
{'ip': '172.253.51.75', 'ci

Программа запускает команду traceroute к введённому адресу и считывает список промежуточных IP-адресов, через которые проходят пакеты. Для каждого IP она запрашивает информацию с сервиса ipinfo.io - страну, провайдера и автономную систему (AS). Затем выводит всё это в виде таблицы. Это помогает увидеть маршрут прохождения данных, понять, через какие узлы и сети они идут, и найти возможные проблемы с сетью. Если узел не отвечает или адрес не разрешается, программа сообщает об этом и может прервать трассировку.